In [1]:
import arcpy
import datetime
import reportlab
from reportlab.pdfgen import canvas
# from reportlab.lib.pagesizes import letter
from PyPDF2 import PdfFileReader, PdfFileWriter

In [2]:
working_folder = r'C:\Users\JR067290\OneDrive - Jacobs\Documents\ArcGIS\Projects\Newport_LCRR'
# Set the workspace (change this to your local path)
gdb = r'C:\Users\JR067290\OneDrive - Jacobs\Documents\ArcGIS\Projects\Newport_LCRR\Newport_LCRR.gdb'

arcpy.env.workspace = gdb
# Feature class and related table paths
wServices = f'{gdb}\\wServices'
wMeters = f'{gdb}\\wMeters'
LCRR_Letter_Tracking = f'{gdb}\\LCRR_Letter_Tracking'

# Get the current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")  # Formats the date as YYYY-MM-DD

# Output PDF path
output_pdf_path = working_folder + 'Postcards\Postcard_' + current_date + '.pdf'
template_pdf_path = f'{working_folder}\\Newport_SLM_Postcard.pdf'

In [ ]:
def create_feature_set(input_feature_class):
    """
    Creates a feature set from the input feature class.
    
    Args:
    input_feature_class (str): The path to the input feature class.
    
    Returns:
    arcpy.FeatureSet: A feature set created from the input feature class.
    """
    # Create a new FeatureSet object
    feature_set = arcpy.FeatureSet()
    
    # Load the feature class into the FeatureSet
    feature_set.load(input_feature_class)
    
    return feature_set

if __name__ == "__main__":
    # Define the input feature class parameter (for example, from tool parameters in ArcGIS Pro)
    input_feature_class = arcpy.GetParameterAsText(0)  # This would be set in the tool's parameters
    
    # Call the create_feature_set function with the input feature class
    output_feature_set = create_feature_set(input_feature_class)
    
    # Set the output parameter to the feature set, so it can be used in the toolbox
    arcpy.SetParameter(1, output_feature_set)

In [4]:

def add_multiline_address_to_pdf(template_pdf_path, output_pdf_path, address_lines, x_position=288, y_position=94, line_spacing=14):
    """
    Adds a multi-line address to a PDF.

    Parameters:
    - template_pdf_path: Path to the input PDF template.
    - output_pdf_path: Path where the modified PDF will be saved.
    - address_lines: A list of strings, each representing a line of the address.
    - x_position: The X coordinate for the start of the address.
    - y_position: The Y coordinate for the start of the address (bottom line).
    - line_spacing: The amount of space between each line of the address.
    """
    # Create a temporary PDF with the address
    address_pdf_path = 'temp_address_overlay.pdf'
    c = canvas.Canvas(address_pdf_path)
    
    # Draw each line of the address, adjusting the Y position for each line
    current_y_position = y_position
    for line in reversed(address_lines):  # Start from the bottom line if y_position is the bottom line
        c.drawString(x_position, current_y_position, line)
        current_y_position += line_spacing  # Move up for the next line
    
    c.save()

    # Merge the address overlay with the original PDF
    original = PdfFileReader(open(template_pdf_path, 'rb'))
    address_overlay = PdfFileReader(open(address_pdf_path, 'rb'))
    output_pdf = PdfFileWriter()

    # Assuming you're adding the address to the first page of the PDF
    page = original.getPage(0)
    page.mergePage(address_overlay.getPage(0))
    output_pdf.addPage(page)

    # If there are more pages, add them too
    for pageNum in range(1, original.numPages):
        output_pdf.addPage(original.getPage(pageNum))
    
    with open(output_pdf_path, 'wb') as out_file:
        output_pdf.write(out_file)
        

In [5]:
# Determine whether to process selected features or all features
use_selected_features = False
if int(arcpy.GetCount_management(wServices)[0]) > 0:
    use_selected_features = True

# Fields to be used from the feature class and related table
feature_fields = ["LetterRelID", "Acctnum"]
table_fields = ["LetterID", "LetterRelID", "LetterSentDate", "LetterType"]
meters_fields = ["acctnum", "owner_address1", "owner_address2", "owner_address3", "owner_address4", "owner_address5"]


In [6]:
# Create a dictionary to map 'acctnum' to owner_address fields from wMeters
acctnum_to_address = {}
with arcpy.da.SearchCursor(wMeters, meters_fields) as cursor:
    for row in cursor:
        # Store all owner_address fields in a list, ensuring they are added only if they are not None or empty
        address_fields = [row[i] for i in range(1, len(row)) if row[i]]
        acctnum_to_address[row[0]] = address_fields


In [7]:
acctnum_to_address

{'B036-17000': ['NEWPORT RESTORATION FOUNDATION',
  '51 TOURO ST',
  'NEWPORT RI 02840'],
 'B036-17500': ['SHUFELT JEAN', '1998 WEST MAIN ROAD', 'MIDDLETOWN RI  02842'],
 'B036-17600': ['SIMMONS BRIAN', '1942 WEST MAIN RD', 'MIDDLETOWN RI 02842'],
 'B036-17695': ['SIMMONS ALEXANDER JR',
  '2028 WEST MAIN RD',
  'MIDDLETOWN RI 02842'],
 'B036-17696': ['SIMMONS BRIAN', '2030 WEST MAIN RD', 'MIDDLETOWN RI 02842'],
 'B037-00010': ['NEWPORT COUNTY HEALTH CLINIC',
  '127 JOHNNY CAKE HILL RD',
  'MIDDLETOWN RI 02842'],
 'B037-00101': ['LONTOC,ANTONIO JR', '76 DEVIN DR', 'PORTSMOUTH RI  02871'],
 'B067-11740': ['MEDEIROS MICHAEL', '221 TAYLOR RD', 'PORTSMOUTH RI 02871'],
 'B067-20035': ['MANNING BRIDGET',
  '55 LAWTON BROOK LN',
  'PORTSMOUTH RI  02871'],
 'B067-20037': ['GRUNAWALT RICHARD',
  'C/O KURT GRUNAWALT',
  '212 PARIS VALLEY',
  'VACAVILLE CA 95687'],
 'B067-20026': ['THOMPSON JENNIFER',
  '46 LAWTON BROOK LN UNIT D',
  'PORTSMOUTH RI 02871'],
 'B067-20039': ['RAMOS MEAGAN & CURTIS',

In [18]:
add_multiline_address_to_pdf(template_pdf_path, output_pdf_path, address_lines)

In [5]:
# Determine whether to process selected features or all features
feature_count = arcpy.GetCount_management(wServices)
selection_count = int(feature_count.getOutput(0))
use_selected_features = False

# Check if there are selected features
if selection_count < int(arcpy.GetCount_management(wServices)[0]):
    use_selected_features = True

# Fields to be used from the feature class and related table
feature_fields = ["LetterRelID", "PointAddress"]
table_fields = ["LetterRelID", "LetterSentDate", "LetterType"]

In [6]:
# Start an edit session and operation
with arcpy.da.Editor(arcpy.env.workspace) as editor:
    # Create a SearchCursor for the feature class
    with arcpy.da.SearchCursor(wServices, feature_fields, None if use_selected_features else "LetterRelID IS NOT NULL") as cursor:
        # Open an InsertCursor to add records to the related table
        with arcpy.da.InsertCursor(LCRR_Letter_Tracking, table_fields) as insert_cursor:
            for row in cursor:
                # Create a new row for the related table
                new_row = (row[0], current_date, "Postcard")  # Include LetterRelID, current date, and set LetterType to "Postcard"
                # Insert the new row into the related table
                insert_cursor.insertRow(new_row)

                 # Draw address label on PDF
                address = row[1]
                draw_label(x, y, address)
                x += label_width + x_gap
                if x + label_width > width - left_margin:
                    x = left_margin
                    y -= label_height + y_gap
                    if y < top_margin:
                        c.showPage()
                        y = height - top_margin - label_height

# Save the PDF
c.save()


print("Operation completed: Records added and PDF generated.")

Operation completed: Records added and PDF generated.


In [ ]:
def generate_pdf(output_pdf, cursor, label_width, label_height, x_gap, y_gap, top_margin, left_margin):
    # Initialize PDF canvas
    c = canvas.Canvas(output_pdf, pagesize=letter)
    width, height = letter
    x, y = left_margin, height - top_margin - label_height

    for row in cursor:
        # [Your logic to draw labels, handle page breaks, etc.]

    # Save the PDF
    c.save()

In [ ]:
# Use this function to generate the PDF
with arcpy.da.SearchCursor(wServices, feature_fields, None if use_selected_features else "LetterRelID IS NOT NULL") as cursor:
    output_pdf = working_folder + '\Postcard_' + current_date + '.pdf'
    generate_pdf(output_pdf, cursor, label_width, label_height, x_gap, y_gap, top_margin, left_margin)


USING THE SLM Survey Postcard 8x6

In [ ]:
import arcpy
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO

# Paths
arcpy.env.workspace = "C:/path/to/your/workspace"
feature_class = "C:/path/to/your/feature_class"
pdf_template_path = '/path/to/Newport_SLM Survey Postcard 8x6_Dec 15-Draft.pdf'

# Coordinates for the address placement
address_x = 470  # Adjust as needed
address_y = 670  # Adjust as needed

# Function to create a temporary PDF with the address
def create_temp_pdf(address):
    packet = BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    can.drawString(address_x, address_y, address)
    can.save()
    packet.seek(0)
    return packet

# Iterate over the selected features
with arcpy.da.SearchCursor(feature_class, ["Address"]) as cursor:
    for i, row in enumerate(cursor):
        address = row[0]

        # Create a temporary PDF with the address
        temp_pdf_stream = create_temp_pdf(address)

        # Load the template PDF and append the temporary PDF with the address
        final_pdf = arcpy.mp.PDFDocumentOpen(pdf_template_path)
        temp_pdf_path = f"C:/path/to/temp/address_temp_{i}.pdf"
        with open(temp_pdf_path, "wb") as f:
            f.write(temp_pdf_stream.read())

        # Append the page with the address to the template PDF
        final_pdf.appendPages(temp_pdf_path)
        
        # Save the final PDF
        output_pdf_path = f"C:/path/to/output/address_Postcard_{i}.pdf"
        final_pdf.saveAndClose()
        os.remove(temp_pdf_path)  # Clean up the temporary PDF file

        print(f"Saved PDF for address: {address} as {output_pdf_path}")
